In [ ]:
!helm init

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands
from visualizer import get_graph

MINIKUBE_IP=commands.getoutput('minikube ip')

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


In [ ]:
%%bash
cd ../helm-charts
helm install seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false

In [ ]:
!kubectl get pods

In [ ]:
get_graph("resources/model.json")

In [ ]:
!kubectl apply -f resources/model.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath="{.status}"

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
get_graph("resources/random_ab_test.json")

In [ ]:
!kubectl apply -f resources/random_ab_test.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/random_ab_test.json

In [ ]:
get_graph("resources/ensemble.json")

In [ ]:
!kubectl apply -f resources/ensemble.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example-ensembles -o jsonpath='{.status}'

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/ensemble.json

In [ ]:
!helm delete seldon-core --purge